In [542]:
from fastapi import FastAPI
from pydantic import BaseModel
import uvicorn
import nest_asyncio
import sqlite3


In [543]:
class BookItem(BaseModel):
    Book_ISBN: str
    Book_title: str
    Author: str
    Price: int
    Qty: int


In [544]:
db = sqlite3.connect("P_6406021421316.db")
cur = db.cursor()


In [545]:
app = FastAPI()

In [546]:
@app.get("/Books")
async def search_book_all():
    sql = "select * from BookStore"
    res = cur.execute(sql).fetchall()
    return res


In [547]:
@app.get("/Books/{id}")
async def search_book_one(id: str):
    try:
        sql = "select * from BookStore where Book_id = ?"
        res = cur.execute(sql, (id,))
        data = res.fetchone()
        if (data != None):
            return {"status": "ok", "data": data}
        else:
            return {"status": "NotFound", "data": ""}
    except sqlite3.Error as err:
        return {"status": "fail", "error": err}


In [548]:
@app.get("/Books/Name/{Name}")
async def search_book_by_name(Name: str):
    try:
        sql = "select * from BookStore where Book_title = ?"
        res = cur.execute(sql, (Name,))
        data = res.fetchone()
        if (data != None):
            return {"status": "ok", "data": data}
        else:
            return {"status": "NotFound", "data": ""}
    except sqlite3.Error as err:
        return {"status": "fail", "error": err}


In [549]:
@app.post("/Books")
async def insert_book(book: BookItem):
    try:
        sql = '''INSERT INTO BookStore (Book_ISBN,Book_title,Author,Price,Qty) VALUES (?,?,?,?,?)'''
        values = (book.Book_ISBN, book.Book_title,
                  book.Author, book.Price, book.Qty)
        cur.execute(sql, values)
        db.commit()
        return {"status": "insert success"}
    except sqlite3.Error as err:
        return {"status": "fail", "error": err}


In [550]:
@app.put('/Books/{id}')
async def update_book(book: BookItem, id: int):
    try:
        sql = '''UPDATE BookStore SET Book_ISBN = ?,Book_title = ?,Author = ?,Price = ?,Qty = ? WHERE Book_id = ?'''
        values = (book.Book_ISBN, book.Book_title,
                  book.Author, book.Price, book.Qty, id)
        cur.execute(sql, values)
        db.commit()
        return {"status": "edit success"}
    except sqlite3.Error as err:
        return {"status": "fail", "error": err}


In [551]:
@app.delete('/Books/{id}')
async def delete_book(id: str):
    try:
        sql = '''DELETE FROM BookStore WHERE Book_id=?'''
        cur.execute(sql, (id,))
        db.commit()
        return {"status": "delete success"}
    except sqlite3.Error as err:
        return {"status": "fail", "error": err}


In [552]:
@app.get("/Books/Price/{price}/")
async def show_price_with_option(price:int, option: str):
    try:
        print(option)
        if  option == "MORE":
            sql = "select * from BookStore order by Price DESC"
            res = cur.execute(sql)
            data = res.fetchall()
            return {"status": "ok", "data": data}
        elif option == "LESS":
            sql = "select * from BookStore order by Price ASC"
            res = cur.execute(sql)
            data = res.fetchall()
            return {"status": "ok", "data": data}
        elif option == "EQUAL" and price != None:
            sql = "select * from BookStore where Price = ?"
            res = cur.execute(sql, (price,))
            data = res.fetchall()
            return {"status": "ok", "data": data}
        else:
            return {"status": "NotFound", "data": ""}
    except sqlite3.Error as err:
        return {"status": "fail", "error": err}

In [553]:
nest_asyncio.apply()
uvicorn.run(app, port=8080)


INFO:     Started server process [6540]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8080 (Press CTRL+C to quit)


MORE
INFO:     127.0.0.1:59684 - "GET /Books/Price/1/?option=MORE HTTP/1.1" 200 OK
LESS
INFO:     127.0.0.1:59697 - "GET /Books/Price/1/?option=LESS HTTP/1.1" 200 OK
EQUAL
INFO:     127.0.0.1:59705 - "GET /Books/Price/649/?option=EQUAL HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [6540]
